---

**Assignment 3: Mobility Tracking (Connected Mobility Basics, Winter 2019/20)**

Felix Schorer | Haoyang Sun | Simon Zachau

---

<strong id="tocheading">Table of Contents</strong>

<div id="toc"></div>

In [ ]:
%%javascript
// adapted from https://github.com/kmahelona/ipython_notebook_goodies/blob/gh-pages/ipython_notebook_toc.js
var toc = "";
var level = 0;
var levels = {}
$('#toc').html('');

$(":header").each(function(i){
    var titleText = this.innerHTML;
    var openLevel = this.tagName[1];

    if (levels[openLevel]){
        levels[openLevel] += 1;
    } else{
        levels[openLevel] = 1;
    }

    if (openLevel > level) {
        toc += (new Array(openLevel - level + 1)).join('<ul class="toc">');
    } else if (openLevel < level) {
        toc += (new Array(level - openLevel + 1)).join("</ul>");
        for (i=level;i>openLevel;i--){
            levels[i]=0;
        }
    }

    level = parseInt(openLevel);
    
    if (level < 3) {
        if (this.id == ''){
            this.id = this.innerHTML.replace(/ /g,"-")
        }
        var anchor = this.id;

        toc += '<li><a href="#' + encodeURIComponent(anchor) + '">' + titleText + '</a></li>';
    }

});


if (level) {
    toc += (new Array(level + 1)).join("</ul>");
}


$('#toc').append(toc);

---

In [ ]:
%pip install numpy matplotlib scapy[basic] tqdm tabulate

# 1. Introduction

We collected Wifi probes with a Raspberry Pi and 2 antennas on different commutes around Munich to analyse them. 
Specifically, we look at the number of packets and their MAC addresses relative to time, and therefore also location. 
In the beginning, we asked ourselves if we could track devices over multiple days during the time of a route 
and if we can we infer from the data when the train/subway stops.

# 2. Routes

In [ ]:
from tabulate import tabulate


def list_captures(*captures):
    print(tabulate(captures, headers=('File', 'Description', 'Start Date/Time')))

## 2.1 Route from Kaufering to Garching Forschungszentrum

We measured the route from Kaufering to Garching Forschungszentrum on 3 different days each before Christmas and after Christmas. 
It involves the train until Munich Central Station, the S-Bahn from Munich Central Station to Marienplatz, 
and the U6 from Marienplatz to Garching Forschungszentrum. The fact that our measurements involve multiple days, 
but are conducted at 7:00 AM consistently every morning, gives us the chance to compare the results.

![route from Kaufering to Garching Forschungszentrum](./illustrations/kaufering_garchingforschungzentrum.png)

In [ ]:
from captures import commutes

list_captures(*commutes)

## 2.2 Round Trip Routes of U2 and U6

On the 19th of December we measured 1 round trip in each the U2 and the U6 at noon. 
Both lines meet in the city center but cross different areas of Munich. 
Different demographics and destinations might enable different observations.
The U2 round trip was measured in two files (stopped at the first destination), whereas 
the U6 round trip was measured in one file. We make them comparable in the preprocessing
section "3.2 Stitching U2 Time Series Together".

![route of U2 and U6](./illustrations/u2u6.png)

In [ ]:
from captures import u2_to_messestadt, u2_to_feldmoching, u6_roundtrip

list_captures(u2_to_messestadt, u2_to_feldmoching, u6_roundtrip)

## 2.3 Route from Munich Central Station to Dortmund Central Station

On the 24th of December we measured a trip in the ICE from Munich Central Station to Dortmund Central Station. 
We might be able to analyse different devices and device usages compared to subway lines in the city center.

![route from Munich Central Station to Dortmund Central Station](./illustrations/munich_dortmund.png)

In [ ]:
from captures import munich_to_dortmund

list_captures(munich_to_dortmund)

# 3. Preprocessing

## 3.1 Parsing Captures

In [ ]:
import random
import numpy as np
from tqdm import tqdm
from scapy.all import PcapReader
from scapy.layers.dot11 import Dot11ProbeReq


def accumulate(file, *accumulators):
    with PcapReader(file) as reader, tqdm(unit='packets', desc=file) as pbar:
        for packet in reader:
            for accumulator in accumulators:
                accumulator(packet)
            pbar.update()


class TimeSeriesAccumulator(list):
    TYPE = ('timestamp', np.float), ('mac', np.bytes_, 6)
    
    def __call__(self, packet):
        if packet.haslayer(Dot11ProbeReq):
            timestamp = packet.time
            mac = bytes.fromhex(packet.addr2.replace(':', ''))
            self.append((timestamp, mac))
            
    def as_numpy_array(self):
        return np.array(self, dtype=np.dtype([*self.TYPE]))


class RandomProbeSampler(list):
    PROBABILITY = 0.001
    
    def __call__(self, packet):
        if packet.haslayer(Dot11ProbeReq) and random.random() <= self.PROBABILITY:
            self.append(packet)

In [ ]:
from captures import all_captures

time_series_data = dict()  # capture -> time series data
samples = dict()  # capture -> list of probe request samples

for capture in all_captures:
    accumulator, sampler = TimeSeriesAccumulator(), RandomProbeSampler()
    accumulate(capture.filename, accumulator, sampler)
    time_series = accumulator.as_numpy_array()
    
    # timestamps in captures are relative, make them absolute
    timestamp_0 = np.amin(time_series[:]['timestamp'])
    correction = capture.start_date.timestamp() - timestamp_0
    time_series[:]['timestamp'] += correction
    
    time_series_data[capture], samples[capture] = time_series, sampler

## 3.2 Stitching U2 Time Series Together

In [ ]:
from captures import Capture

u2_roundtrip = Capture(
    filename=None, 
    description='U2 round trip Feldmoching/Messestadt Ost',
    start_date=u2_to_messestadt.start_date
)

time_series_data[u2_roundtrip] = np.append(time_series_data[u2_to_messestadt], time_series_data[u2_to_feldmoching])
samples[u2_roundtrip] = samples[u2_to_messestadt] + samples[u2_to_feldmoching]

# 4. Analysis

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from captures import make_legend


def plot_line_chart(data, x_label=None, y_label=None, title=None, x_scale='linear', y_scale='linear'):
    plt.figure(figsize=(18, 6))
    legends = []
    for xs, ys, legend in data:
        plt.plot(xs, ys)
        legends.append(legend)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.xscale(x_scale)
    plt.yscale(y_scale)
    plt.title(title)
    plt.legend(legends)


def plot_stack_plot(data, x_label=None, y_label=None, title=None, x_scale='linear', y_scale='linear'):
    plt.figure(figsize=(18, 6))
    xs, yss, labels = data
    plt.stackplot(xs, yss, labels=labels)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.xscale(x_scale)
    plt.yscale(y_scale)
    plt.title(title)
    plt.legend()

## 4.1 Number of Probes Per Time Interval

In [ ]:
import math


def duration(timestamps):
    return np.amax(timestamps) - np.amin(timestamps)


def frequency_count(timestamps, time_interval):
    number_of_intervals = math.ceil(duration(timestamps) / time_interval)
    counts = np.zeros(number_of_intervals)
    rel_timestamps = timestamps - np.amin(timestamps)
    for timestamp in rel_timestamps:
        counts[math.floor(timestamp / time_interval)] += 1
    time = np.arange(0, len(counts) * time_interval, time_interval)
    return time, counts

In [ ]:
TIME_INTERVAL = 120


def plot_frequency_counts(*captures):
    plots = []

    for capture in captures:
        timestamps = time_series_data[capture][:]['timestamp']
        xs, ys = frequency_count(timestamps, TIME_INTERVAL)
        plots.append((xs, ys, make_legend(capture)))

    plot_line_chart(plots, x_label='time (s)', y_label='number of probes per {}s'.format(TIME_INTERVAL))

### Route from Kaufering to Garching Forschungszentrum

In [ ]:
plot_frequency_counts(*commutes)

#### Observations
- The trip on 12/18/2019 (green line) ends early due to the battery running out.
- The change in the number of packets is consistent in relative terms.
- The number of packets per 120s starts at about 500 in Kaufering, grows until Central Station until it reaches 2000 packets per 120s, peaks multiple times around the city center, and decreases until the destination Garching Forschungszentrum.

### Round Trip Routes of U2 and U6

In [ ]:
plot_frequency_counts(u2_roundtrip, u6_roundtrip)

#### Observations
Todo

### Route from Munich Central Station to Dortmund Central Station

In [ ]:
plot_frequency_counts(munich_to_dortmund)

#### Observations
Todo

## 4.2 Inter-Contact Times of Mac Addresses

In [ ]:
from collections import defaultdict


def compute_inter_contact_times(probes):
    min_max_timestamps = defaultdict(tuple)  # mac -> min, max timestamp
    for probe in probes:
        current_min_max = min_max_timestamps[probe['mac']]
        min_timestamp = min([*current_min_max, probe['timestamp']])
        max_timestamp = max([*current_min_max, probe['timestamp']])
        min_max_timestamps[probe['mac']] = min_timestamp, max_timestamp
    return sorted(max_timestamp - min_timestamp for min_timestamp, max_timestamp in min_max_timestamps.values())


def compute_cdf(values):
    total = len(values)
    accumulator = 0
    ys, xs = [], []
    current_value = None
    for value in values:
        if current_value != value and current_value is not None:
            ys.append(accumulator)
            xs.append(current_value)
        current_value = value
        accumulator += 1 / total
    return xs, ys

In [ ]:
def plot_inter_contact_times(*captures):
    plots = []

    for capture in captures:
        inter_contact_times = compute_inter_contact_times(time_series_data[capture])
        xs, ys = compute_cdf(inter_contact_times)
        plots.append((xs, ys, make_legend(capture)))
    
    plot_line_chart(plots, x_label='time (s)', y_label='per cent', x_scale='log')

In [ ]:
plot_inter_contact_times(*commutes, u2_roundtrip, u6_roundtrip, munich_to_dortmund)

#### Observations
- Almost 90% of MAC addresses are seen for less than 0.1 seconds.
 - Probably due to MAC address randomization

## 4.3 Vendors

In [ ]:
!wget http://standards-oui.ieee.org/oui/oui.csv

In [ ]:
import csv


with open('oui.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader)
    vendors = {bytes.fromhex(row[1]): row[2] for row in reader}


def lookup_vendor(mac):
    return vendors.get(mac[:3], '<randomized>')

In [ ]:
from collections import Counter


def vendor_frequency_count(probes, time_interval, top=10):
    total = Counter(lookup_vendor(mac) for mac in probes[:]['mac'])
    ranks = {vendor: index for index, (vendor, _) in enumerate(total.most_common(top))}
    
    timestamps = probes[:]['timestamp']
    number_of_intervals = math.ceil(duration(timestamps) / time_interval)
    counts = np.zeros((top + 1, number_of_intervals))
    min_timestamp = np.amin(timestamps)
    for probe in probes:
        vendor = lookup_vendor(probe['mac'])
        rank = ranks.get(vendor, top)
        rel_timestamp = probe['timestamp'] - min_timestamp
        counts[rank, math.floor(rel_timestamp / time_interval)] += 1
    time = np.arange(0, number_of_intervals * time_interval, time_interval)
    return time, counts, [*ranks.keys(), '<other>']


def compute_vendor_distribution(*args, **kwargs):
    time, counts, labels = vendor_frequency_count(*args, **kwargs)
    return time, counts / np.sum(counts, axis=0), labels

In [ ]:
def plot_vendor_distributions(*captures):
    for capture in captures:
        vendor_distribution = compute_vendor_distribution(time_series_data[capture], TIME_INTERVAL, top=10)
        plot_stack_plot(vendor_distribution, 
                        x_label='time (s)', y_label='percentage of probes per {}s'.format(TIME_INTERVAL), 
                        title=make_legend(capture))

### Route from Kaufering to Garching Forschungszentrum

In [ ]:
plot_vendor_distributions(*commutes)

#### Observations
- The majority of MAC addresses is randomized.
- The Raspberry Pi is sending probe requests as well.
- The conductor of the measurements owns a OnePlus phone.

### Round Trip Routes of U2 and U6

In [ ]:
plot_vendor_distributions(u2_roundtrip, u6_roundtrip)

#### Observations
Todo

### Route from Munich Central Station to Dortmund Central Station

In [ ]:
plot_vendor_distributions(munich_to_dortmund)

#### Observations
Todo